In [ ]:
class DistributedMultiDataLoaders:
    def __init__(self, dataset1, dataset2, dataset3, batch_size,generator, num_workers, distributed=False):
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.distributed = distributed
        self.pin_memory=True
class DistributedMultiDataLoaders:
    def __init__(self, dataset1, dataset2, dataset3, batch_size,generator, num_workers, distributed=False):
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.distributed = distributed
        self.pin_memory=True
        self.generator=generator

        # Create DistributedSampler if in distributed mode
        if self.distributed:
            self.sampler1 = DistributedSampler(dataset1)
            self.sampler2 = DistributedSampler(dataset2)
            self.sampler3 = DistributedSampler(dataset3)
        else:
            self.sampler1 = None
            self.sampler2 = None
            self.sampler3 = None

        # Create DataLoader instances
        self.dataloader1 = DataLoader(
            dataset1,
            batch_size=batch_size,
            num_workers=num_workers,
            sampler=self.sampler1,
            pin_memory=self.pin_memory,
            generator=self.generator,
            shuffle=False
        )

        self.dataloader2 =GeometricDataLoader(
            dataset2,
            batch_size=batch_size,
            num_workers=num_workers,
            sampler=self.sampler2,
            pin_memory=self.pin_memory,
            generator=self.generator,
            shuffle=False
        )

        self.dataloader3 = DataLoader(
            dataset3,
            batch_size=batch_size,
            num_workers=num_workers,
            sampler=self.sampler3,
            pin_memory=self.pin_memory,
            generator=self.generator,
            shuffle=False
        )

        # Wrap DataLoader with DistributedDataParallel if in distributed mode
        '''
        if self.distributed:
            self.dataloader1 = DistributedDataParallel(self.dataloader1)
            self.dataloader2 = DistributedDataParallel(self.dataloader2)
            self.dataloader3 = DistributedDataParallel(self.dataloader3)
        '''
    def set_shuffle(self, shuffle):
        self.dataloader1.shuffle = shuffle
    def set_epoch(self, epoch):
        self.dataloader1.sampler.set_epoch(epoch)
        self.dataloader2.sampler.set_epoch(epoch)
        # 迭代所有DataLoader实例
        self.iter1 = iter(self.dataloader1)
        self.iter2 = iter(self.dataloader2)
        self.iter3 = iter(self.dataloader3)
        return self

    def __next__(self):
        # 并行获取来自每个DataLoader的数据
        try:
            batch1 = next(self.iter1)
            batch2 = next(self.iter2)
            batch3 = next(self.iter3)
        except StopIteration:
            # 重新设置采样器，以便下一个epoch可以重新开始
            self.sampler1.set_epoch(self.sampler1.epoch + 1)
            self.sampler2.set_epoch(self.sampler2.epoch + 1)
            self.sampler3.set_epoch(self.sampler3.epoch + 1)
            raise StopIteration

        return batch1, batch2, batch3
